# 第9章 深層学習によって時系列を扱う仕組みを知ろう(1-5節)
ここでは、RNNとCNNによる時系列データの学習について学んでいきます。  

In [ ]:
#Colaboratory環境の設定
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MathProgramming/Chapter9

In [ ]:
#ライブラリの設定
!pip install -q -r ./requirements.txt

## 9-2 RNNを使ってsin波を予測してみよう


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import random
import os

#シード値の固定
seed = 1
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
%matplotlib inline

#データの読み込み
data = np.loadtxt("./rnn_sin_40_80.csv")
plt.plot(data[:500])
plt.show()

In [ ]:
#history_stepsステップ数を入力に使い、future_stepsステップ数を予測する様にデータを加工する
def create_dataset(data, history_steps, future_steps):
    input_data = []
    output_data= []
    
    for i in range(len(data)-history_steps-future_steps):
        input_data.append([[val] for val in data[i:i+history_steps]])
        output_data.append(data[i+history_steps:i+history_steps+future_steps])
    
    return np.array(input_data), np.array(output_data)

train_data = data[:int(len(data) * 0.75)]
test_data = data[int(len(data) * 0.75):]

#10ステップ分のデータから５ステップ未来までを予測するようなデータを作成する
history_steps = 10
future_steps = 5
x_train, y_train = create_dataset(train_data, history_steps, future_steps)
x_test, y_test   = create_dataset(test_data, history_steps, future_steps)  

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.optimizers import Adam

#モデルの構築
model_rnn = Sequential()  
model_rnn.add(SimpleRNN(units=future_steps, input_shape=(history_steps,1),return_sequences=False))
model_rnn.add(Dense(future_steps,activation="linear"))  
model_rnn.compile(optimizer = Adam(lr=0.001), loss="mean_squared_error",)

#モデルの構造を表示する
print(model_rnn.summary())

#学習開始
history = model_rnn.fit(x_train, y_train, batch_size=32, epochs=500, verbose=1)

## 9-3 RNNで予測したsin波の評価をしてみよう

In [ ]:
#学習したモデルで予測をする
y_pred = model_rnn.predict(x_test)

plt.figure(figsize=(15,5))
#青色で予測値、オレンジ色で実際の値を表示
plt.plot([p[0] for p in y_pred],color="blue",label="pred")
plt.plot([p[0] for p in y_test],color="orange",label="actual")
plt.legend()
plt.show()

In [ ]:
#学習済みモデルと予測に用いる値、実際の値の３つを引数に取る
def predict_all(model, x_test, y_test):

    #最初のhistory_steps分だけ利用する。
    x_tmp = x_test[0]
    pred_result = []
    for index in range(len(y_test)):
        #x_tmpに入っているデータで予測をする
        pred = model.predict(np.array([x_tmp]))

        #x_tmpの最初のデータを削除して、末尾に今回予測したデータの最初の一つを追加する
        x_tmp = np.append(x_tmp[1:,], pred[0][0].reshape(1,1), axis=0)
        pred_result.append(pred[0][0])
        
    plt.figure(figsize=(15,5))
    #予測した値をと実際の値を表示する
    plt.plot(pred_result,color="blue", label="pred")
    plt.plot([p[0] for p in y_test],color="orange", label="actual")
    plt.legend()
    plt.show()

#予測開始
predict_all(model_rnn, x_test, y_test)

## 9-4 CNNでもsin波の予測をしてみよう


In [ ]:
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, MaxPooling1D, GlobalMaxPooling1D

#モデルの構築
model_conv = Sequential()
model_conv.add(Conv1D(filters=64, kernel_size=4, strides=1, padding='same', input_shape=x_train.shape[1:], activation='relu'))
model_conv.add(Conv1D(filters=128, kernel_size=4,strides=1, padding='same', activation='relu'))
model_conv.add(GlobalMaxPooling1D())
model_conv.add(Dense(future_steps, activation='tanh'))
model_conv.compile(optimizer = Adam(lr=0.001), loss='mean_squared_error')

#モデルの構造を表示する
print(model_conv.summary())

#予測開始
history = model_conv.fit(x_train, y_train, batch_size=32, epochs=500, verbose=1)

In [ ]:
#予測開始
y_pred = model_conv.predict(x_test)

plt.figure(figsize=(15,5))
plt.plot([p[0] for p in y_pred],color="blue",label="pred")
plt.plot([p[0] for p in y_test],color="orange",label="actual")
plt.legend()
plt.show()

In [ ]:
predict_all(model_conv, x_test, y_test)

## 9-5 sin波の予測精度を高めよう


In [ ]:
history_steps_v2 = 40
future_steps_v2 = 10

#40ステップ分のデータから10ステップ未来までを予測するようなデータを作成する
x_train_v2, y_train_v2 = create_dataset(train_data, history_steps_v2, future_steps_v2)
x_test_v2, y_test_v2   = create_dataset(test_data, history_steps_v2, future_steps_v2)  


print(x_train_v2.shape)
print(y_train_v2.shape)

In [ ]:
#RNNのモデル構築
model_rnn_v2 = Sequential()  
model_rnn_v2.add(SimpleRNN(units=future_steps_v2, input_shape=(history_steps_v2,1),return_sequences=False))
model_rnn_v2.add(Dense(future_steps_v2,activation="linear"))  
model_rnn_v2.compile(optimizer = Adam(lr=0.001), loss="mean_squared_error",)

#モデルの構造を表示する
print(model_rnn_v2.summary())
        
#学習開始
history = model_rnn_v2.fit(x_train_v2, y_train_v2, batch_size=32, epochs=500, verbose=1)

In [ ]:
predict_all(model_rnn_v2, x_test_v2, y_test_v2)

In [ ]:
#CNNのモデルを構築
model_conv_v2 = Sequential()
model_conv_v2.add(Conv1D(filters=64, kernel_size=4, strides=1, padding='same', input_shape=x_train_v2.shape[1:], activation='relu'))
model_conv_v2.add(Conv1D(filters=128, kernel_size=4,strides=1, padding='same', activation='relu'))
model_conv_v2.add(GlobalMaxPooling1D())
model_conv_v2.add(Dense(future_steps_v2, activation='tanh'))
model_conv_v2.compile(optimizer = Adam(lr=0.001), loss='mean_squared_error')

#モデルの構造を表示
print(model_conv_v2.summary())

#学習開始
history = model_conv_v2.fit(x_train_v2, y_train_v2, batch_size=32, epochs=500, verbose=1)

In [ ]:
predict_all(model_conv_v2, x_test_v2, y_test_v2)